In [1]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop,Adam
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os
from tensorflow.keras.applications import VGG16, ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D,Concatenate,Dropout
from tensorflow.keras.models import Model
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
import zipfile

zip_path = "/content/drive/MyDrive/chest_xray.zip"
extract_path = "/content/chest_xray_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [3]:
import os

for folder in os.listdir(extract_path):
    print("📂", folder)


📂 test
📂 train


In [5]:


# Paths
base_dir = "/content/chest_xray_data"
train_dir = os.path.join(base_dir, "train")
test_dir = os.path.join(base_dir, "test")

# Image Generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # we'll split here

train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training',
    shuffle=True
)

val_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation',
    shuffle=True
)


import tensorflow as tf

# Helper to wrap generator output with proper structure
def duplicate_input(generator):
    for x, y in generator:
        yield (x, x), y  # Create 2 inputs for ensemble


# Define output_signature here
output_signature = (
    (
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)
    ),
    tf.TensorSpec(shape=(None,), dtype=tf.float32)
)



# Create dual input test set
test_dual_input = tf.data.Dataset.from_generator(
    lambda: duplicate_input(test_gen),
    output_signature=(
        (
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32)
        ),
        tf.TensorSpec(shape=(None,), dtype=tf.float32)
    )
)
# Wrap the generator with tf.data.Dataset
train_dual_input = tf.data.Dataset.from_generator(
    lambda: duplicate_input(train_gen),
    output_signature=output_signature
)

val_dual_input = tf.data.Dataset.from_generator(
    lambda: duplicate_input(val_gen),
    output_signature=output_signature
)



# Data Augmentation only on training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.15  # you can adjust this if needed
)


train_gen = train_datagen.flow_from_directory(
    '/content/chest_xray_data/train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)



test_datagen = ImageDataGenerator(rescale=1./255)

test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

# Base models
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in vgg.layers:
    layer.trainable = False

for layer in resnet.layers:
    layer.trainable = False

# Custom heads
vgg_out = GlobalAveragePooling2D()(vgg.output)
resnet_out = GlobalAveragePooling2D()(resnet.output)


combined = Concatenate()([vgg_out, resnet_out])
combined = Dropout(0.5)(combined)  # Dropout added here after concatenation
combined = Dense(256, activation='relu')(combined)
combined = Dropout(0.5)(combined)  # Dropout after Dense layer
output = Dense(1, activation='sigmoid')(combined)


model = Model(inputs=[vgg.input, resnet.input], outputs=output)

# Compile
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Training
# Note: We'll use two separate generators for VGG and ResNet if needed, but this model assumes same input
model.fit(
    train_dual_input,
    validation_data=val_dual_input,
    steps_per_epoch=len(train_gen),
    validation_steps=len(val_gen),
    epochs=30
)

# Evaluation
model.evaluate(test_dual_input, steps=len(test_gen))

# Save model
# model.save('ensemble_cxr_model.keras')
import joblib

# Assume model is a scikit-learn model or any picklable object
joblib.dump(model, "ensemble_cxr_model.pkl")



Found 4187 images belonging to 2 classes.
Found 1045 images belonging to 2 classes.
Found 4448 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Epoch 1/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 156s 924ms/step - accuracy: 0.6864 - loss: 0.6828 - val_accuracy: 0.7426 - val_loss: 0.5013
Epoch 2/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 112s 807ms/step - accuracy: 0.7202 - loss: 0.5837 - val_accuracy: 0.7703 - val_loss: 0.4542
Epoch 3/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 108s 779ms/step - accuracy: 0.7478 - loss: 0.5195 - val_accuracy: 0.8383 - val_loss: 0.4116
Epoch 4/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.7666 - loss: 0.4820 - val_accuracy: 0.8727 - val_loss: 0.3895
Epoch 5/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.7930 - loss: 0.4383 - val_accuracy: 0.8938 - val_loss: 0.3568
Epoch 6/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.8037 - loss: 0.4164 - val_accuracy: 0.8861 - val_loss: 0.3668
Epoch 7/30
139/139 ━━━━━━━━━━━━━━━━━━━━ 112s 808ms

['ensemble_cxr_model.pkl']

In [4]:
# prompt: code to change the .pkl to .keras file
import joblib
from tensorflow import keras

# Load the model from the .pkl file
loaded_model = joblib.load("/content/ensemble_cxr_model (1).pkl")

# Save the model as a .keras file
loaded_model.save('ensemble_cxr_new_model.keras')
